In [1]:
# Pruning - Distillation Knowledge - Weight Sharing - Quantization all the Llama3 series models

In [1]:
from typing import List 
import fire 

import torch.distributed as dist 

from foxer import Llama 